In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')
import numpy as np
import pandas as pd
from scipy import stats
from itertools import product
from matplotlib import pyplot as plt
%matplotlib inline

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
MERGE_TABLE_STORE_4600 = pd.read_csv('/content/gdrive/My Drive/A Reinforcement Learning Approach for Inventory Optimization in Retail/MERGE_TABLE_STORE_4600/MERGE_TABLE_STORE_4600.csv', sep=';', decimal='.')
sl_data = pd.read_csv('/content/gdrive/My Drive/A Reinforcement Learning Approach for Inventory Optimization in Retail/MERGE_TABLE_STORE_4600/data/echelon/echelon_1_sl.csv', sep=';')
empty_states = pd.read_csv('/content/gdrive/My Drive/A Reinforcement Learning Approach for Inventory Optimization in Retail/MERGE_TABLE_STORE_4600/empty_states.csv', index_col=0)
not_empty_states = pd.read_csv('/content/gdrive/My Drive/A Reinforcement Learning Approach for Inventory Optimization in Retail/MERGE_TABLE_STORE_4600/not_empty_states.csv', index_col=0)
demand_data = pd.read_csv('/content/gdrive/My Drive/A Reinforcement Learning Approach for Inventory Optimization in Retail/MERGE_TABLE_STORE_4600/demand_data.csv', index_col=0)

In [0]:
class SMART():
    def __init__(self, ss_data=None, sl_data=None, demand_data=None, pairs_data=None, max_steps=10, lt=1, alpha_order=0.3, alpha=0.01, probability=0.7):
        self.ss_data = ss_data
        self.sl_data = sl_data
        self.demand_data = demand_data
        self.pairs_data = pairs_data
        self.max_steps = max_steps
        self.lt = lt
        self.alpha_order = alpha_order
        self.alpha = alpha
        self.probability = probability

    def load_ss_data(self, data):
        data['Timestamp'] = pd.to_datetime(data['curr_date'], format='%d%b%Y')
        data['curr_date'] = data['Timestamp']
        data = data.set_index(['Timestamp'], drop=True)
        self.ss_data = data
        return self

    def load_sl_data(self, data):
        data['date_from'] = pd.to_datetime(data['date_from'], format='%d%b%Y')
        data['date_to'] = pd.to_datetime(data['date_to'], format='%d%b%Y')
        self.sl_data = data
        return self

    def load_demand_data(self, data):
        self.demand_data = data
        return self

    def load_pairs_data(self, data):
        self.pairs_data = data
        return self

    def initial_state(self, shop_id, product_id):
        state = pd.DataFrame(columns=['location', 'sku', 'sales', 'stock', 'sl', 'order'])

        store_sales_t = self.ss_data[self.ss_data['store_id']==shop_id]
        store_sales_t = store_sales_t[store_sales_t['product_id']==product_id].fillna(0)
        store_sales_t = store_sales_t.drop(columns=['curr_date', 'flg_spromo'])

        sl_data_t = self.sl_data.set_index(['date_from'], drop=True)
        sl_data_t = sl_data_t[sl_data_t['location_ids']==shop_id]
        sl_data_t = sl_data_t[sl_data_t['product_ids']==product_id]

        if not sl_data_t.empty:
            all_days = pd.date_range(start=sl_data_t.index.min(), end=sl_data_t['date_to'].max())
            sl_data_t.index = pd.DatetimeIndex(sl_data_t.index)
            sl_data_t = sl_data_t.reindex(all_days, method='ffill')
            sl_data_t = sl_data_t.drop(columns=['date_to'])

            state_t = pd.merge(store_sales_t, sl_data_t['value'], how='right', left_on=store_sales_t.index, right_on=sl_data_t.index)
            state_t = state_t.rename(columns={'store_id': 'location', 'product_id': 'sku', 's_qty': 'sales', 'value': 'sl'})
            state_t = state_t.drop(columns=['key_0'])
            state_t = state_t.drop_duplicates()
            state_t = state_t.dropna()
            state_t['order'] = 0
            state = state.append(state_t, ignore_index=True)
        return state

    def calculate_states(self):
        shops_ss = np.sort(self.ss_data['store_id'].unique())
        products_ss = np.sort(self.ss_data['product_id'].unique())

        shops_sl = np.sort(self.sl_data['location_ids'].unique())
        products_sl = np.sort(self.sl_data['product_ids'].unique())
        
        empty_states = pd.DataFrame(columns=['shop_id', 'product_id'])
        not_empty_states = pd.DataFrame(columns=['shop_id', 'product_id'])

        for shop_id in [shop for shop in shops_ss if shop in shops_sl]:
            for product_id in [prod for prod in products_ss if prod in products_sl]:
                state = self.initial_state(product_id, shop_id)
                if state.empty:
                    empty_states = empty_states.append({'shop_id': shop_id, 'product_id': product_id}, ignore_index=True)
                else:
                    not_empty_states = not_empty_states.append({'shop_id': shop_id, 'product_id': product_id}, ignore_index=True)
        return not_empty_states, empty_states

    def demand_restoring(self, shop_id, product_id):
        iv_ts = self.ss_data.reset_index().groupby('Timestamp').agg({'stock':np.max})
        iv_ts = iv_ts.reindex(pd.date_range(np.min(iv_ts.index), np.max(iv_ts.index))).fillna(method='ffill')

        sales_ts = self.ss_data[(self.ss_data['product_id'] == product_id) & (self.ss_data['store_id'] == shop_id)]
        iv_sales = sales_ts[['s_qty']].merge(iv_ts, how='right', left_index=True, right_index=True)
        
        positive_iv_sales = iv_sales[iv_sales.max(axis=1) > 0]
        life_start_date = positive_iv_sales.index[0]
        life_end_date = positive_iv_sales.index[-1]
        iv_sales = iv_sales[(iv_sales.index >= life_start_date) & (iv_sales.index <= life_end_date)] 

        zero_idx = (iv_sales['stock'] == 0) & (iv_sales['s_qty'] == 0)
        sales_equal_inv_idx = (iv_sales['stock'] == iv_sales['s_qty']) 
        sales_greater_i_idx = (iv_sales['stock'] <= iv_sales['s_qty']) 

        demand_data = pd.DataFrame(columns=['shop_id', 'product_id', 'lambda', 'demand'])

        iv_sales['weights'] = [1 for x in iv_sales.index]
        iv_sales['scalar'] = iv_sales['s_qty'][(~zero_idx)] * iv_sales['weights'][(~zero_idx)]
        sum_k = iv_sales['scalar'].sum()
        n_k_less_m = iv_sales['weights'][(~zero_idx) & (~sales_greater_i_idx)].sum()
        n_k_equal_m = iv_sales['weights'][(~zero_idx) & sales_greater_i_idx].sum()

        # alpha=100 #!!!!!!!!!
        alpha = 1
        LAMBDA = sum_k / (n_k_less_m + alpha * n_k_equal_m)

        max_sales = self.ss_data[(self.ss_data['store_id']==shop_id) &
                                 (self.ss_data['product_id']==product_id)]['s_qty'].max()
        demand = max(stats.poisson.ppf(0.99, LAMBDA), max_sales, 1)
        demand_data = demand_data.append({'shop_id': shop_id, 'product_id': product_id, 'lambda': LAMBDA, 'demand': demand}, ignore_index=True)
        demand_data[['shop_id', 'product_id', 'demand']] = demand_data[['shop_id', 'product_id', 'demand']].astype('int')
        return demand_data

    def calculate_demand(self):
        demand_data = pd.DataFrame(columns=['shop_id', 'product_id', 'lambda', 'demand'])
        for index, row in self.pairs_data.iterrows():
            shop_id = row['shop_id']
            product_id = row['product_id']
            demand_data = demand_data.append(self.demand_restoring(shop_id, product_id), ignore_index=True)
        return demand_data

    def product_condition(self, state_range, OUL, ROL):
        prods = []
        for prod in list(product(state_range, OUL, ROL)):
            if prod[1] > prod[2]:
                prods.append(prod)
        return prods

    def initial_action(self, shop_id, product_id, state_range):
        action = pd.DataFrame(columns=['location', 'sku', 'state', 'OUL', 'ROL'])

        max_demand = int(self.demand_data[(self.demand_data['product_id'] == product_id) & (self.demand_data['shop_id'] == shop_id)].iloc[0].demand)
        OUL = [i for i in range(max_demand + 1)]
        ROL = [i for i in range(max(OUL))]
        
        action_t = pd.DataFrame(self.product_condition(state_range, OUL, ROL), columns=['state', 'OUL', 'ROL'])
        action = pd.merge(action, action_t, how='outer')
        values = {'location': shop_id, 'sku': product_id}
        action = action.fillna(value=values)
        action['R'] = 0.0
        return action

    def sales_update(self, data, shop_id, product_id):
        sales = max(self.demand_data[(self.demand_data['shop_id']==shop_id) & (self.demand_data['product_id']==product_id)].iloc[0].demand,
                    data.iloc[-1].stock + data.iloc[-self.lt].order)
        return sales

    def stock_update(self, data, shop_id, product_id):
        stock = max(data.iloc[-1].stock + data.iloc[-self.lt].order - self.demand_data[(self.demand_data['shop_id']==shop_id) & (self.demand_data['product_id']==product_id)].iloc[0].demand, 0)
        return stock

    def order_update(self, data, shop_id, product_id, OUL, ROL):
        if data.iloc[-1].stock <= ROL:
            rec_order = max(OUL - data.iloc[-1].stock - data.iloc[-1].order, 0)
        else:
            rec_order = 0
        order = np.random.binomial(n=rec_order, p=self.alpha_order)
        return order

    def fit_single(self, shop_id, product_id):
        state_data = self.initial_state(shop_id, product_id)
        action_data = self.initial_action(shop_id, product_id, np.arange(len(state)))
        env_data = pd.DataFrame(columns=['location', 'sku', 'order', 'sales', 'stock'])

        for starting_state in range(len(state_data)):
            m = 0 # time step
            T = 0 # total time
            C = 0 # cumulative reward / total reward
            p = 0 # reward rate / average reward
            
            while m < self.max_steps:
                if m == 0:
                    i = starting_state
                    alpha = self.alpha
                    probability = 1

                    env_data = env_data.append({'location': shop_id,
                                                'sku': product_id,
                                                'order': state_data.at[i, 'order'],
                                                'sales': state_data.at[i, 'sales'],
                                                'stock': state_data.at[i, 'stock']},
                                               ignore_index=True)
                else:
                    alpha = self.alpha / m
                    probability = self.probability / m

                non_exploratoty = np.random.binomial(n=1, p=1-probability)
                if non_exploratoty:
                    action_index = action_data[action_data['state']==i].R.idxmax()
                else:
                    action_index = action_data[action_data['state']==i].R.idxmax()
                    if len(action_data[action_data['state']==i]) > 1:
                        action_index = np.random.choice(action_data[(action_data['state']==i) &
                                                                    (action_data.index != action_index)].index, 1)[0]

                env_data = env_data.append({'location': shop_id,
                                            'sku': product_id,
                                            'order': self.order_update(env_data, shop_id, product_id, action_data.loc[action_index].OUL, action_data.loc[action_index].ROL),
                                            'sales': self.sales_update(env_data, shop_id, product_id),
                                            'stock': self.stock_update(env_data, shop_id, product_id)},
                                           ignore_index=True)

                new_state = pd.DataFrame(columns=['location', 'sku', 'sales', 'stock', 'sl', 'order'])
                new_state = new_state.append({'location': shop_id,
                                              'sku': product_id,
                                              'sales': env_data.iloc[-1].sales,
                                              'stock': env_data.iloc[-1].stock,
                                              'sl': state_data.at[i, 'sl'],
                                              'order': env_data.iloc[-1].order,
                                              }, ignore_index=True)

                if state_data[(state_data.sales==new_state.sales[0]) &
                              (state_data.stock==new_state.stock[0]) &
                              (state_data.sl==new_state.sl[0]) &
                              (state_data.order==new_state.order[0])].empty:
                    state_data = state_data.append(new_state, ignore_index=True)
                    j = state_data.index.max()
                    action_data = action_data.append(self.initial_action(shop_id, product_id, [j]), ignore_index=True)
                else:
                    j = state_data[(state_data.sales==new_state.sales[0]) &
                                   (state_data.stock==new_state.stock[0]) &
                                   (state_data.sl==new_state.sl[0]) &
                                   (state_data.order==new_state.order[0])].index[0]

                r = (new_state['sales'] - new_state['stock'] * (1 - new_state['sl']) / new_state['sl']).sum()
                action_data.at[action_index, 'R'] = (1 - alpha) * action_data.at[action_index, 'R'] + alpha * max(r - p * self.lt + action_data[action_data['state']==j].R.max(), 0)

                if non_exploratoty:
                    C = C + r
                    T += self.lt
                    p = C / T
                i = j
                m += 1

        state_data[['sales', 'stock']] = state_data[['sales', 'stock']].astype('int')
        env_data = env_data.astype('int')
        return state_data, action_data, env_data

    def fit_multiple(self):
        state_data = pd.DataFrame(columns=['location', 'sku', 'sales', 'stock', 'sl', 'order'])
        action_data = pd.DataFrame(columns=['location', 'sku', 'state', 'OUL', 'ROL', 'R'])
        env_data = pd.DataFrame(columns=['location', 'sku', 'order', 'sales', 'stock'])

        for index, row in self.pairs_data.loc[:1].iterrows():
        # for index, row in self.pairs_data.iterrows():
            shop_id = row['shop_id']
            product_id = row['product_id']

            state = self.initial_state(shop_id, product_id)
            action = self.initial_action(shop_id, product_id, np.arange(len(state)))
            env = pd.DataFrame(columns=['location', 'sku', 'order', 'sales', 'stock'])

            for starting_state in range(len(state)):
                m = 0 # time step
                T = 0 # total time
                C = 0 # cumulative reward / total reward
                p = 0 # reward rate / average reward
                
                while m < self.max_steps:
                    if m == 0:
                        i = starting_state
                        alpha = self.alpha
                        probability = 1

                        env = env.append({'location': shop_id,
                                          'sku': product_id,
                                          'order': state.at[i, 'order'],
                                          'sales': state.at[i, 'sales'],
                                          'stock': state.at[i, 'stock']},
                                         ignore_index=True)
                    else:
                        alpha = self.alpha / m
                        probability = self.probability / m
                    
                    non_exploratoty = np.random.binomial(n=1, p=1-probability)
                    if non_exploratoty:
                        action_index = action[action['state']==i].R.idxmax()
                    else:
                        action_index = action[action['state']==i].R.idxmax()
                        if len(action[action['state']==i]) > 1:
                            action_index = np.random.choice(action[(action['state']==i) &
                                                                   (action.index != action_index)].index, 1)[0]
                    
                    env = env.append({'location': shop_id,
                                      'sku': product_id,
                                      'order': self.order_update(env, shop_id, product_id, action.loc[action_index].OUL, action.loc[action_index].ROL),
                                      'sales': self.sales_update(env, shop_id, product_id),
                                      'stock': self.stock_update(env, shop_id, product_id)},
                                     ignore_index=True)

                    new_state = pd.DataFrame(columns=['location', 'sku', 'sales', 'stock', 'sl', 'order'])
                    new_state = new_state.append({'location': shop_id,
                                                  'sku': product_id,
                                                  'sales': env.iloc[-1].sales,
                                                  'stock': env.iloc[-1].stock,
                                                  'sl': state.at[i, 'sl'],
                                                  'order': env.iloc[-1].order,
                                                  }, ignore_index=True)

                    if state[(state.sales==new_state.sales[0]) &
                             (state.stock==new_state.stock[0]) &
                             (state.sl==new_state.sl[0]) &
                             (state.order==new_state.order[0])].empty:
                        state = state.append(new_state, ignore_index=True)
                        j = state.index.max()
                        action = action.append(self.initial_action(shop_id, product_id, [j]), ignore_index=True)
                    else:
                        j = state[(state.sales==new_state.sales[0]) &
                                  (state.stock==new_state.stock[0]) &
                                  (state.sl==new_state.sl[0]) &
                                  (state.order==new_state.order[0])].index[0]

                    r = (new_state['sales'] - new_state['stock'] * (1 - new_state['sl']) / new_state['sl']).sum()
                    action.at[action_index, 'R'] = (1 - alpha) * action.at[action_index, 'R'] + alpha * max(r - p * self.lt + action[action['state']==j].R.max(), 0)

                    if non_exploratoty:
                        C = C + r
                        T += self.lt
                        p = C / T
                    i = j
                    m += 1

            state_data = state_data.append(state, ignore_index=True)
            action_data = action_data.append(action, ignore_index=True)
            env_data = env_data.append(env, ignore_index=True)

        state_data[['sales', 'stock']] = state_data[['sales', 'stock']].astype('int')
        env_data = env_data.astype('int')
        return state_data, action_data, env_data

In [0]:
model = SMART()

model.load_ss_data(MERGE_TABLE_STORE_4600)
model.load_sl_data(sl_data)
model.load_demand_data(demand_data)
model.load_pairs_data(not_empty_states)

# not_empty_states, empty_states = model.calculate_states()
# demand_data = model.calculate_demand()

# demand = model.demand_restoring(4600, 25100)
# state = model.initial_state(4600, 25100)
# action = model.initial_action(4600, 25100, np.arange(len(state)))

state_data, action_data, env_data = model.fit_single(4600, 25100)
# state_data, action_data, env_data = model.fit_multiple()

In [102]:
state_data

,location,sku,sales,stock,sl,order
0,4600,25100,0,3,0.95,0
1,4600,25100,0,3,0.90,0
2,4600,25100,0,5,0.81,0
3,4600,25100,0,5,0.95,0
4,4600,25100,1,5,0.95,0
5,4600,25100,0,4,0.95,0
6,4600,25100,0,4,0.90,0
7,4600,25100,3,1,0.95,0
8,4600,25100,2,0,0.95,0
9,4600,25100,2,0,0.95,2


In [103]:
action_data

,location,sku,state,OUL,ROL,R
0,4600,25100,0,1,0,0.000000
1,4600,25100,0,2,0,0.000000
2,4600,25100,0,2,1,0.029474
3,4600,25100,1,1,0,0.000000
4,4600,25100,1,2,0,0.000000
...,...,...,...,...,...,...
58,4600,25100,19,2,0,0.000000
59,4600,25100,19,2,1,0.020147
60,4600,25100,20,1,0,0.000000
61,4600,25100,20,2,0,0.000000


In [104]:
env_data

,location,sku,order,sales,stock
0,4600,25100,0,0,3
1,4600,25100,0,3,1
2,4600,25100,0,2,0
3,4600,25100,2,2,0
4,4600,25100,0,2,0
...,...,...,...,...,...
72,4600,25100,0,2,0
73,4600,25100,0,2,0
74,4600,25100,0,2,0
75,4600,25100,1,2,0
